# Converting PyTorch to TensorFlow Lite for xCORE Using ONNX

ONNX is an open format built to represent machine learning models. We can convert from PyTorch to ONNX, then from ONNX to TensorFlow, then from TensorFlow to TensorFlow Lite, and finally, run it through xformer to optimise it for xCORE.

Ensure that you have installed Python 3.8 and have the installed requirements.txt

In [ ]:
import sys
import os

# allow importing helper functions from local module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import PyTorch Model

For this example, we use mobilenet_v2.

In [ ]:
import torch

pytorch_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# Switch the model to eval mode
pytorch_model.eval()

### Download the Class Labels

In [ ]:
import requests
resp = requests.get("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt")
# Read the categories
categories = [s.strip() for s in resp.text.splitlines()]

### Perform an infrence on the pytorch model
Perform inference on the model to see how it works.

In [ ]:
# Download an image to test against
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# We will test and train with these params
batch_size = 1
channels = 3
height = 224
width = 224

In [ ]:
from PIL import Image
from torchvision import transforms

# Open testing image
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(height),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Note Pytorch is BCHW
input_tensor = preprocess(input_image)

In [ ]:
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

with torch.no_grad():
    output = pytorch_model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Show top categories per image
vals, idxs = torch.topk(probabilities, 5)
pytorch_results = [(categories[idx], prob) for (idx, prob) in zip(idxs.tolist(), vals.tolist())]
for cat, prob in  pytorch_results:
    print(cat, ':', prob)

## Convert to ONNX


In [ ]:
# This is only for shape info for tracing the model during conversion
sample_input = torch.rand((batch_size, channels, height, width))

onnx_model_path = "mobilenet_v2.onnx"

torch.onnx.export(
    pytorch_model,
    sample_input,
    onnx_model_path,
    input_names=['image'],
    output_names=['probabilities']
)

### Check the exported model

In [ ]:
import onnx
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

### Check ONNX Output

In [ ]:
import onnxruntime 
import numpy as np
ort_session = onnxruntime.InferenceSession(onnx_model_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def softmax(xs):
    return np.exp(xs)/sum(np.exp(xs))

# compute ONNX Runtime output prediction
input_batch_np = to_numpy(input_batch)
ort_inputs = {ort_session.get_inputs()[0].name: input_batch_np}
ort_outs = ort_session.run(None, ort_inputs)

# The input is still BCHW
data = zip(range(len(ort_outs[0][0])), softmax(ort_outs[0][0]))

onnx_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  onnx_results:
    print(cat, ':', prob)

### Convert ONNX to Keras
We do this using the `onnx2tf` package: https://github.com/PINTO0309/onnx2tf

In [ ]:
import onnx2tf

keras_model_path = 'mobilenet_v2.tf'

keras_model = onnx2tf.convert(
    input_onnx_file_path=onnx_model_path,
    output_folder_path=keras_model_path,
    non_verbose=True,
)

### Check the conversion to keras

In [ ]:
#transpose the input_batch into BHWC order for tensorflow
tf_input_data = np.transpose( input_batch.numpy(), [0, 2, 3, 1])

keras_output_data = keras_model(tf_input_data)

probs = softmax(keras_output_data[0])
data = zip(range(len(probs)), probs)
keras_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  keras_results:
    print(cat, ':', prob)

## Convert Keras to TFLite (float32)

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32

tflite_model = converter.convert()

# Save the model.
tflite_float_model_path = 'mobilenet_v2_float.tflite'
with open(tflite_float_model_path, 'wb') as f:
  f.write(tflite_model)

### Check it Worked

In [ ]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_float_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

# Convert PyTorch Input Tensor into Numpy Matrix and Reshape for TensorFlow
# (Pytorch model expects C x H x W but TF expects H x W x C)
tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_input_data)
tfl_interpreter.invoke()

tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

probs = softmax(tfl_output_data[0])
data = zip(range(len(probs)), probs)
tfl_float32_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  tfl_float32_results:
    print(cat, ':', prob)

## Convert Keras to TFLite (int8)
We will still feed the data into the model in float32 format for convinence but the internals of the model will be int8. This will require representitive data but as we interface in float32 we can use the pytorch preprocessing. 

### Representative Dataset

To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to [Converting a keras model into an xcore optimised tflite model](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb) for more details on this.

#### Download & Extract Images from Data Set

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

ds = tfds.load('imagenette', split='train', as_supervised=True, shuffle_files=True).shuffle(1000).batch(1).prefetch(10).take(1000)

# Iterate over the sampled images and preprocess them
def representative_dataset():
    for image, _ in ds:
        pil_img = tf.keras.utils.array_to_img(image[0])
        pytorch_batch = preprocess(pil_img).unsqueeze(0)
        tf_batch = np.transpose(pytorch_batch.numpy(), [0, 2, 3, 1])
        yield [tf_batch]

In [ ]:
# Now do the conversion to int8
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32

tflite_int8_model = converter.convert()

# Save the model.
tflite_int8_model_path = 'mobilenet_v2.tflite'
with open(tflite_int8_model_path, 'wb') as f:
  f.write(tflite_int8_model)

In [ ]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_int8_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

# Convert PyTorch Input Tensor into Numpy Matrix and Reshape for TensorFlow
tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_input_data)
tfl_interpreter.invoke()

tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

probs = softmax(tfl_output_data[0])
data = zip(range(len(probs)), probs)
tfl_int8_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  tfl_int8_results:
    print(cat, ':', prob)

# Analysing Models

Defined below is a function to print out the operator counts of a model.

In [ ]:
import utils

In [ ]:
# Let's inspect the int8 model
utils.print_operator_counts(tflite_int8_model)

## Compare Accuracy

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import requests
from typing import List

# load dataset
ds, info = tfds.load('imagenet_v2', split='test', with_info=True, as_supervised=True, shuffle_files=False)
ds = ds.shuffle(100, reshuffle_each_iteration=True)    

In [ ]:
def accuracy_tflite(top_n:int = 1, samples=1000, verbose=False) -> float:
    if top_n < 1 or n > 1000:
        raise ValueError
    
    # take subset of dataset
    selection = ds.prefetch(10).take(samples)

    correct = 0
    incorrect = 0
    
    for image, label in selection: 
        trueCatIdx = tf.get_static_value(label)
        # convert to PIL.Image
        img = tf.keras.utils.array_to_img(image)
        
        # preprocess using PyTorch functions then convert back into Tf.Tensor
        pytorch_batch = preprocess(img).unsqueeze(0)
        tf_batch = np.transpose( pytorch_batch.numpy(), [0, 2, 3, 1])

        # use same tflite interpreter as before
        tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_batch)
        tfl_interpreter.invoke()

        output = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

        # Sort into List[Tuple[index, confidence]] ordered by confidence (descending)
        data = sorted(
            zip(range(len(output[0])), output[0]),
            key=lambda x: x[1], reverse=True
        )

        top_n_results: List[int] = [idx for (idx, _) in data[:top_n]]

        if trueCatIdx in top_n_results:
            correct = correct + 1
        else:
            incorrect = incorrect + 1
            if(verbose):
                print("--incorrect--")
                print(f"True Category: {categories[trueCatIdx]}({trueCatIdx})")
                print([f"Top-{top_n} categories: {categories[idx]}({idx})" for idx in top_n_results])
                display(img)
    
    accuracy = (correct / (correct+incorrect))
    print(f"Top-{top_n} accuracy (TFLite Model): {accuracy * 100}% ({correct}/{correct+incorrect})")
    return accuracy

def accuracy_torch(top_n: int = 1, samples=1000, verbose=False):
    if top_n < 1 or n > 1000:
        raise ValueError
    
    # take subset of dataset
    selection = ds.prefetch(10).take(samples)

    correct = 0
    incorrect = 0
    
    for image, label in selection: 
        trueCatIdx = tf.get_static_value(label)
        
        # convert to PIL.Image
        img = tf.keras.utils.array_to_img(image)
        input_batch = preprocess(img).unsqueeze(0)
        
        with torch.no_grad():
            output = pytorch_model(input_batch)

        # Show top categories per image
        vals, idxs = torch.topk(output[0], top_n)
        
        if trueCatIdx in idxs:
            correct = correct + 1
        else:
            incorrect = incorrect + 1
            if(verbose):
                print("--incorrect--")
                print(f"True Category: {categories[trueCatIdx]}({trueCatIdx})")
                print([f"Top-{top_n} categories: {categories[idx]}({idx})" for idx in top_n_results])
                display(img)
    
    accuracy = (correct / (correct+incorrect))
    print(f"Top-{top_n} accuracy (PyTorch Model): {accuracy * 100}% ({correct}/{correct+incorrect})")
    return accuracy

In [ ]:
samples = 500
for n in range(5):
    accuracy_torch(n+1, samples)
    accuracy_tflite(n+1, samples)